<img src="./images/02-api.png" width=700px>


see _get_client.py

In [1]:
import os
import json
import requests
from dotenv import load_dotenv
from pprint import pprint

# we now use OpenAI to send requests
from openai import OpenAI

# import openai

In [2]:
# for console formatting I use the Rich library
from rich.console import Console

console = Console()

In [3]:
# We are familiar with a standard API request
# This is a public API that returns a random joke
# The API developers have a very deterministic response even if random
# Both sides program imperatively

get_random_joke_internet = requests.get(
    "https://official-joke-api.appspot.com/random_joke"
)
print(get_random_joke_internet.json())

{'type': 'general', 'setup': 'What did the scarf say to the hat?', 'punchline': 'You go on ahead, I am going to hang around a bit longer.', 'id': 183}


In [4]:
def get_llm_client(llm_choice):
    if llm_choice == "GROQ":
        client = OpenAI(
            base_url="https://api.groq.com/openai/v1",
            api_key=os.environ.get("GROQ_API_KEY"),
        )
        return client
    elif llm_choice == "OPENAI":
        load_dotenv()  # load environment variables from .env fil
        client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
        return client
    else:
        raise ValueError("Invalid LLM choice. Please choose 'GROQ' or 'OPENAI'.")

In [5]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

LLM_CHOICE = "OPENAI"
LLM_CHOICE = "GROQ"

if OPENAI_API_KEY:
    print(f"OPENAI_API_KEY exists and begins {OPENAI_API_KEY[:14]}...")
else:
    print("OPENAI_API_KEY not set")

if GROQ_API_KEY:
    print(f"GROQ_API_KEY exists and begins {GROQ_API_KEY[:14]}...")
else:
    print("GROQ_API_KEY not set")


client = get_llm_client(LLM_CHOICE)
if LLM_CHOICE == "GROQ":
    MODEL = "llama-3.3-70b-versatile"
else:
    MODEL = "gpt-4o-mini"

print(f"LLM_CHOICE: {LLM_CHOICE} - MODEL: {MODEL}")

OPENAI_API_KEY exists and begins sk-proj-vIt-L1...
GROQ_API_KEY exists and begins gsk_0yKDCuUXkz...
LLM_CHOICE: GROQ - MODEL: llama-3.3-70b-versatile


Essentially, we build up data to pass as SYSTEM MESSAGE that sets the context for the conversation and contexctual data for the LLM.

I split this into core SYSTEM_MESSAGE and PROMPT_ENGINEERING = system_message

We then create the USER PROMPT that is the input to the LLM.

OpenAI was trained on this structure, so it is most effective to do it this way.


In [6]:
# We now create our ENDPOINT but on the CLIENT side and in NATURAL LANGUAGE

system_message = """
You are an assistant that is great at telling jokes.
"""

# Here is where we can do some prompt engineering - we are adding to the system message and creating our endpoint as it were.


# We can also load a system message from a file if we want to keep it separate
with open("./prompts/01_api_sample_prompt.md", "r") as f:
    prompt_engineering = f.read()
    print("Reading prompt engineering from file ./prompts/01_api_sample_prompt.md")
    # print(prompt_engineering)


# We add the prompt engineering to the system message
system_message = prompt_engineering


# We create our user prompt and will add all of these to the payload.
user_prompt = "Tell a light-hearted joke for a general audience"

Reading prompt engineering from file ./prompts/01_api_sample_prompt.md


In [7]:
# We create a list of messages to pass to the LLM

# OpenAI works was trained with a list of messages - system, user, assistant - so using this is most effective

prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt},
]
# we will get back as we have seen previously an 'assistant' message.

In [8]:
response = client.chat.completions.create(
    model=MODEL, messages=prompts, temperature=0.7
)
print(response.choices[0].message.content)
output = response.choices[0].message.content.replace("\n", "")

{"setup": "Why couldn't the bicycle stand up by itself?", 
"punchline": "Because it was two-tired", 
"rating": 9, 
"next": "PUBLISH", 
"explanation": "This joke is funny because it plays on the common phrase 'two-tired' meaning exhausted, but also references the two tires on a bicycle"}


For the application, we could create a state object so that we can track the state of the application - it is custom.


In [9]:
state = {"next": "", "setup": "", "punchline": "", "rating": ""}

In [10]:
print(output)

{"setup": "Why couldn't the bicycle stand up by itself?", "punchline": "Because it was two-tired", "rating": 9, "next": "PUBLISH", "explanation": "This joke is funny because it plays on the common phrase 'two-tired' meaning exhausted, but also references the two tires on a bicycle"}


In [11]:
# We can update the state object for use in our app...
result = json.loads(output)
if result["next"] == "PUBLISH":
    state["next"] = result["next"]
    state["setup"] = result["setup"]
    state["rating"] = result["rating"]
    state["punchline"] = result["punchline"]
else:
    state["next"] = "RETRY"
pprint(state)

{'next': 'PUBLISH',
 'punchline': 'Because it was two-tired',
 'rating': 9,
 'setup': "Why couldn't the bicycle stand up by itself?"}


In [12]:
# We can extract the NEXT step from the Autonomous AI Agent. This could be app based or we could invoke another agent.

# This can then be used in a range of sofware design patterns for program flow as in a ROUTER (04) or even a FILTER of data needed for the FAQ (03) example - e.g. {"category": "sprints"} with the DB searched for relevant 'sprint' information.

# This illustrates the AUTONOMY aspect of AI Agents in that it determines the next part of the workflow.

console.print(
    f"Using {LLM_CHOICE} and {MODEL} -> NEXT STEP: [dark_orange bold]{state['next']}[/]"
)

Using GROQ and llama-3.3-70b-versatile -> NEXT STEP: PUBLISH